# 基于mindspore实现强化学习的示例

在强化学习中，Q-learning是一种基于值(values-based)的算法，最终的return是一个表格，即Q-Table。

这个表格的每一行都代表着一个状态（state），每一行的每一列都代表着一个动作（action），而每个值就代表着如果在该state下，采取该action所能获取的最大的未来期望奖励。

通过Q-Table就可以找到每个状态下的最优行为，进而通过找到所有的最优action来最终得到最大的期望奖励。

本节参考一个“treasure on right”的游戏游戏来实现Q-Learning这个算法。

游戏介绍：宝藏treasure在最右边的位置。然后训练一个智能体Agent去获得这个宝藏

参数解释：

N_STATES ：表示状态数量。其实就是位置数量

EPSILON： ϵ-greedy 的ϵ  

同理，ALPHA，GAMMA也类似

MAX_EPISODES ： 表示玩多少轮游戏来训练

FRESH_TIME：用来输出的参数，控制多久刷新一次页面之类

TerminalFlag： 用来记录游戏结束的标志符，方便统一，就放在外面


In [1]:
import time

import numpy as np
import pandas as pd

N_STATES = 6
ACTIONS = ["left", "right"]
EPSILON = 0.9
ALPHA = 0.1
GAMMA = 0.9
MAX_EPISODES = 15
FRESH_TIME = 0.3
TerminalFlag = "terminal"


def build_q_table(n_states, actions):
    return pd.DataFrame(
        np.zeros((n_states, len(actions))),
        columns=actions
    )


def choose_action(state, q_table):
    state_table = q_table.loc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_table.idxmax()
    return action_name


def get_env_feedback(S, A):
    if A == "right":
        if S == N_STATES - 2:
            S_, R = TerminalFlag, 1
        else:
            S_, R = S + 1, 0
    else:
        S_, R = max(0, S - 1), 0
    return S_, R


def update_env(S, episode, step_counter):
    env_list = ["-"] * (N_STATES - 1) + ["T"]
    if S == TerminalFlag:
        interaction = 'Episode %s: total_steps = %s' % (episode + 1, step_counter)
        print(interaction)
        time.sleep(2)
    else:
        env_list[S] = '0'
        interaction = ''.join(env_list)
        print(interaction)
        time.sleep(FRESH_TIME)


def rl():
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)
            q_predict = q_table.loc[S, A]

            if S_ != TerminalFlag:
                q_target = R + GAMMA * q_table.loc[S_, :].max()
            else:
                q_target = R
                is_terminated = True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)
            S = S_
            update_env(S, episode, step_counter + 1)
            step_counter += 1
    return q_table


if __name__ == '__main__':
    q_table = rl()
    print(q_table)



0----T
-0---T
0----T
-0---T
--0--T
---0-T
----0T
Episode 1: total_steps = 7
0----T
0----T
0----T
-0---T
--0--T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
--0--T
-0---T
0----T
-0---T
0----T
-0---T
--0--T
---0-T
----0T
Episode 2: total_steps = 20
0----T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 3: total_steps = 8
0----T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 4: total_steps = 7
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 5: total_steps = 6
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 6: total_steps = 6
0----T
-0---T
--0--T
-0---T
--0--T
---0-T
----0T
Episode 7: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
---0-T
----0T
Episode 8: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
Episode 9: total_steps = 5
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 10: total_steps = 6
0----T
-0---T
--0--T
---0-T
----0T
Episode 11: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 12: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 13: to